Periodontitis Detection: Proof of Concept

This notebook serves as an initial proof of concept for detecting periodontitis using image enhancement, tooth segmentation, and bone loss measurement. 

We will test the following steps on a single image:
1. Preprocessing – Apply K-CFDO image enhancement.
2. Tooth Segmentation – Train YOLOv8 segmentation model.

In [1]:
# 1. Preprocessing – Apply K-CFDO image enhancement.
"""
K-CFDO (Kernel-based Contrast and Feature Detail Optimization) Image Enhancement

Parameters:
-----------
ρ (rho): A regularization parameter in the range [0,1] that controls the balance between
         noise reduction and detail preservation. Lower values preserve more details but may
         retain more noise, while higher values reduce noise but might blur fine details.
         - When ρ approaches 0: Emphasizes detail preservation
         - When ρ approaches 1: Emphasizes noise reduction

k: A scaling factor that adjusts the strength of the kernel filter. It controls the 
   emphasis on image features and the smoothing effects. Typically a value > 1.
   - Smaller k values: Result in stronger enhancement effects
   - Larger k values: Produce more subtle enhancements

r: Represents the normalized pixel intensity value in the range [0,1], which is treated
   as a probability value in the context of the algorithm. Each pixel's intensity is
   transformed according to its value and the parameters.

E_k^(1,1-ρ/k)(r): Represents a special function (related to the Mittag-Leffler function)
                  that is approximated using the Gamma function in this implementation.
                  The Gamma function (Γ) helps in calculating the denominator part of the 
                  enhancement equation.

The algorithm enhances each pixel according to the formula:
Enhanced_value = Original_value × [r^((1-ρ)/k) / Γ(2-ρ/k)]

This transformation:
1. Increases contrast in regions with significant intensity variations
2. Preserves edges and fine details based on the parameter settings
3. Reduces noise in homogeneous regions
4. Improves overall visibility of features in medical images, particularly X-rays
"""
import os
import shutil
import numpy as np
import cv2
from tqdm import tqdm
from scipy import special

# K-CFDO Enhancement Function
def k_cfdo_enhancement(image, rho=0.5, k=1.5):
    img = image.astype(np.float64)
    if img.max() > 1.0:
        img = img / 255.0  # Normalize
    n, m = img.shape
    enhanced_img = np.zeros_like(img)
    for i in range(n):
        for j in range(m):
            r = img[i, j]
            numerator = r ** ((1 - rho) / k)
            denominator = special.gamma(2 - rho / k)
            enhanced_img[i, j] = img[i, j] * (numerator / denominator)
    enhanced_img = (enhanced_img - enhanced_img.min()) / (enhanced_img.max() - enhanced_img.min())
    return (enhanced_img * 255).astype(np.uint8)

# Function to process images and move labels
def process_and_move_data(set_name):
    """
    Process images using K-CFDO enhancement and move corresponding labels.
    
    Parameters:
    -----------
    set_name : str
        Name of the dataset split (train, valid, or test).
    """
    # Define paths
    input_image_dir = f"../data/{set_name}/images"
    input_label_dir = f"../data/{set_name}/labels"
    
    # Define the output paths to be outside of the notebook folder
    output_image_dir = f"../enhanced_data/{set_name}/images"
    output_label_dir = f"../enhanced_data/{set_name}/labels"
    
    # Create output directories if they do not exist
    os.makedirs(output_image_dir, exist_ok=True)
    os.makedirs(output_label_dir, exist_ok=True)
    
    # Process images
    for filename in tqdm(os.listdir(input_image_dir), desc=f"Processing {set_name} images"):
        if filename.endswith((".jpg", ".png", ".jpeg")):
            img_path = os.path.join(input_image_dir, filename)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            
            # Apply enhancement
            enhanced_img = k_cfdo_enhancement(img, rho=0.5, k=1.5)
            
            # Save the enhanced image
            cv2.imwrite(os.path.join(output_image_dir, filename), enhanced_img)
            
            # Move the corresponding label file
            label_file = filename.replace('.jpg', '.txt').replace('.png', '.txt').replace('.jpeg', '.txt')
            label_path = os.path.join(input_label_dir, label_file)
            if os.path.exists(label_path):
                shutil.copy(label_path, os.path.join(output_label_dir, label_file))

# Process each dataset split
for split in ["train", "valid", "test"]:
    process_and_move_data(split)

# Create data.yaml file for YOLOv8
data_yaml_content = f"""path: ./enhanced_data
train: train/images
val: valid/images
test: test/images

nc: 1
names: ['Individual Tooth']
"""

# Save the data.yaml file in the correct directory
yaml_path = "../enhanced_data/data.yaml"
with open(yaml_path, "w") as f:
    f.write(data_yaml_content)

print("✅ Enhancement complete! Final dataset stored in '../enhanced_data/'.")
print("✅ Labels moved successfully.")
print(f"✅ 'data.yaml' created at {yaml_path}.")


Processing test images: 100%|████████████████████████████████████████████████████████████| 5/5 [00:45<00:00,  9.02s/it]

✅ Enhancement complete! Final dataset stored in '../enhanced_data/'.
✅ Labels moved successfully.
✅ 'data.yaml' created at ../enhanced_data/data.yaml.


In [2]:
!pip install ultralytics

In [23]:
# 2. Tooth Segmentation – Train a YOLOv8 segmentation model.
''' 
    This script trains a YOLOv8 segmentation model to automatically segment individual teeth from dental X-ray images. 
    The model is designed for instance segmentation, which detects objects (teeth) and predicts their pixel-wise masks. 
    Using a dataset of labeled X-rays, the model learns to identify and separate each tooth.
    Key training parameters like optimizer, learning rate, and momentum are adjusted for optimal performance.

'''
from ultralytics import YOLO

# Load the YOLOv8 instance segmentation model (the segmentation model)
model = YOLO("yolov8n-seg.pt")  # Make sure you use the segmentation variant

# Train the model on the instance segmentation dataset
model.train(
    data="C:/Users/pauli/DeepDent/periodontitis/enhanced_data/data.yaml",  # Path to the data.yaml file
    epochs=50,                        # Number of epochs
    batch=8,                          # Batch size
    device="cpu",                     # Use CPU for training (or "cuda" if you have a GPU)
    
    # Optimization parameters
    optimizer="AdamW",                # Use AdamW optimizer (default is Adam)
    lr0=0.001,                        # Initial learning rate                        # Final learning rate
    momentum=0.937,                    # Momentum for the optimizer
    weight_decay=0.0005,               # Regularization term
    warmup_epochs=3,                  # Number of epochs for warmup
    warmup_momentum=0.8,              # Momentum during warmup
    warmup_bias_lr=0.1                # Bias learning rate during warmup
)

Ultralytics 8.3.99  Python-3.9.13 torch-2.6.0+cpu CPU (AMD Ryzen 5 5500U with Radeon Graphics)
engine\trainer: task=segment, mode=train, model=yolov8n-seg.pt, data=C:/Users/pauli/DeepDent/periodontitis/enhanced_data/data.yaml, epochs=50, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=cpu, workers=8, project=None, name=train6, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_cro

train: Scanning C:\Users\pauli\DeepDent\periodontitis\enhanced_data\train\labels.cache... 35 images, 0 backgrounds, 0 c
val: Scanning C:\Users\pauli\DeepDent\periodontitis\enhanced_data\valid\labels.cache... 10 images, 0 backgrounds, 0 cor

module 'matplotlib.cm' has no attribute 'register_cmap'


optimizer: AdamW(lr=0.001, momentum=0.937) with parameter groups 66 weight(decay=0.0), 77 weight(decay=0.0005), 76 bias(decay=0.0)
TensorBoard: model graph visualization added 
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs\segment\train6
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


       1/50         0G      1.689      4.112      3.187       1.23        169        640: 100%|██████████| 5/5 [00:36<0
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP

...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP

WARNING  Limiting validation plots to first 50 items per image for speed...
WARNING  Limiting validation plots to first 50 items per image for speed...
WARNING  Limiting validation plots to first 50 items per image for speed...
WARNING  Limiting validation plots to first 50 items per image for speed...
WARNING  Limiting validation plots to first 50 items per image for speed...
WARNING  Limiting validation plots to first 50 items per image for speed...
WARNING  Limiting validation plots to first 50 items per image for speed...
WARNING  Limiting validation plots to first 50 items per image for speed...
WARNING  Limiting validation plots to first 50 items per image for speed...
WARNING  Limiting validation plots to first 50 items per image for speed...


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP


                   all         10        270      0.974      0.981      0.993      0.707      0.983      0.981      0.994       0.62
WARNING  ConfusionMatrix plot failure: module 'matplotlib.cm' has no attribute 'register_cmap'
WARNING  ConfusionMatrix plot failure: module 'matplotlib.cm' has no attribute 'register_cmap'
Speed: 9.0ms preprocess, 78.0ms inference, 0.0ms loss, 53.5ms postprocess per image
Results saved to runs\segment\train6


ultralytics.utils.metrics.SegmentMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000022161B488B0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)', 'Precision-Recall(M)', 'F1-Confidence(M)', 'Precision-Confidence(M)', 'Recall-Confidence(M)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.0410

In [54]:
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt
import os
import sys

sys.path.append('..')  # Add parent directory to path
from src.preprocessing import enhance_image_from_user

# Function to get the latest 'best.pt' model
def get_latest_model(model_dir):
    """
    Get the path to the latest 'best.pt' model in the specified directory.
    
    Parameters:
    -----------
    model_dir : str
        Directory where the YOLO models are stored.
        
    Returns:
    --------
    str
        Path to the latest 'best.pt' model.
    """
    # List all directories in the given path
    runs_dirs = [d for d in os.listdir(model_dir) if os.path.isdir(os.path.join(model_dir, d))]
    
    # Find the latest run folder by looking at the timestamp
    latest_run = max(runs_dirs, key=lambda d: os.path.getmtime(os.path.join(model_dir, d)))
    
    # Define the path to the best.pt model in the latest run
    model_path = os.path.join(model_dir, latest_run, 'weights', 'best.pt')
    
    return model_path

# Step 1: Enhance the image
input_image_path = '../trial.jpg'  # Replace with your input image path
output_dir = '../trial test results'  # Replace with your desired output directory

# Enhance the image - you can also customize rho and k parameters here
enhance_image_from_user(input_image_path, output_dir, rho=0.5, k=1.5)

# Get the path to the enhanced image
enhanced_image_path = os.path.join(output_dir, os.path.basename(input_image_path))

# Step 2: Get the latest model path
model_dir = "C:/Users/pauli/DeepDent/periodontitis/notebooks/runs/segment"  # Your base model directory
latest_model_path = get_latest_model(model_dir)

# Load the YOLOv8 segmentation model
model = YOLO(latest_model_path)

# Run inference on the enhanced image
results = model(enhanced_image_path)

# Display the image with segmentation results 
for result in results:
    
    result.show(labels=False, conf=False)  # No labels, no confidence scores
    
    # Optionally, save the output without bounding boxes and labels
    result.save(filename="output.jpg", labels=False, conf=False)

# Optionally, load and display the saved image using OpenCV
output_image = cv2.imread("output.jpg")
plt.imshow(cv2.cvtColor(output_image, cv2.COLOR_BGR2RGB))
plt.axis("off")
plt.show()

Enhanced image saved to ../trial test results\trial.jpg

image 1/1 C:\Users\pauli\DeepDent\periodontitis\notebooks\..\trial test results\trial.jpg: 320x640 22 Individual Tooths, 69.7ms
Speed: 2.0ms preprocess, 69.7ms inference, 18.8ms postprocess per image at shape (1, 3, 320, 640)


<Figure size 640x480 with 1 Axes>

In [8]:
# 2. Tooth Segmentation – Train a YOLOv8 segmentation model. 
# Without Optimization
''' 
    This script trains a YOLOv8 segmentation model to automatically segment individual teeth from dental X-ray images. 
    The model is designed for instance segmentation, which detects objects (teeth) and predicts their pixel-wise masks. 
    Using a dataset of labeled X-rays, the model learns to identify and separate each tooth.
    Key training parameters like optimizer, learning rate, and momentum are adjusted for optimal performance.

'''
from ultralytics import YOLO

# Load the YOLOv8 instance segmentation model (the segmentation model)
model = YOLO("yolov8n-seg.pt")  # Make sure you use the segmentation variant

# Train the model on the instance segmentation dataset
model.train(
    data="C:/Users/pauli/DeepDent/periodontitis/enhanced_data/data.yaml",  # Path to the data.yaml file
    epochs=50,                        # Number of epochs
    batch=8,                          # Batch size
    device="cpu",                     # Use CPU for training (or "cuda" if you have a GPU)
)

Ultralytics 8.3.99  Python-3.9.13 torch-2.6.0+cpu CPU (AMD Ryzen 5 5500U with Radeon Graphics)
engine\trainer: task=segment, mode=train, model=yolov8n-seg.pt, data=C:/Users/pauli/DeepDent/periodontitis/enhanced_data/data.yaml, epochs=50, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=cpu, workers=8, project=None, name=train12, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_cro

train: Scanning C:\Users\pauli\DeepDent\periodontitis\enhanced_data\train\labels.cache... 35 images, 0 backgrounds, 0 c
val: Scanning C:\Users\pauli\DeepDent\periodontitis\enhanced_data\valid\labels.cache... 10 images, 0 backgrounds, 0 cor


module 'matplotlib.cm' has no attribute 'register_cmap'
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 66 weight(decay=0.0), 77 weight(decay=0.0005), 76 bias(decay=0.0)
TensorBoard: model graph visualization added 
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs\segment\train12
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


       1/50         0G      2.017      4.032      3.574      1.426        169        640: 100%|██████████| 5/5 [00:36<0
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP

                   all         10        270     0.0413      0.459     0.0338     0.0139      0.035      0.389     0.0271    0.00973


...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP

WARNING  Limiting validation plots to first 50 items per image for speed...
WARNING  Limiting validation plots to first 50 items per image for speed...
WARNING  Limiting validation plots to first 50 items per image for speed...
WARNING  Limiting validation plots to first 50 items per image for speed...
WARNING  Limiting validation plots to first 50 items per image for speed...
WARNING  Limiting validation plots to first 50 items per image for speed...
WARNING  Limiting validation plots to first 50 items per image for speed...
WARNING  Limiting validation plots to first 50 items per image for speed...
WARNING  Limiting validation plots to first 50 items per image for speed...
WARNING  Limiting validation plots to first 50 items per image for speed...


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP


                   all         10        270      0.981      0.967      0.993      0.711      0.978      0.963      0.992      0.591
WARNING  ConfusionMatrix plot failure: module 'matplotlib.cm' has no attribute 'register_cmap'
WARNING  ConfusionMatrix plot failure: module 'matplotlib.cm' has no attribute 'register_cmap'
Speed: 1.4ms preprocess, 88.3ms inference, 0.0ms loss, 45.8ms postprocess per image
Results saved to runs\segment\train12


ultralytics.utils.metrics.SegmentMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x00000200916251C0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)', 'Precision-Recall(M)', 'F1-Confidence(M)', 'Precision-Confidence(M)', 'Recall-Confidence(M)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.0410

In [9]:
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt
import os
import sys

sys.path.append('..')  # Add parent directory to path
from src.preprocessing import enhance_image_from_user

# Function to get the latest 'best.pt' model
def get_latest_model(model_dir):
    """
    Get the path to the latest 'best.pt' model in the specified directory.
    
    Parameters:
    -----------
    model_dir : str
        Directory where the YOLO models are stored.
        
    Returns:
    --------
    str
        Path to the latest 'best.pt' model.
    """
    # List all directories in the given path
    runs_dirs = [d for d in os.listdir(model_dir) if os.path.isdir(os.path.join(model_dir, d))]
    
    # Find the latest run folder by looking at the timestamp
    latest_run = max(runs_dirs, key=lambda d: os.path.getmtime(os.path.join(model_dir, d)))
    
    # Define the path to the best.pt model in the latest run
    model_path = os.path.join(model_dir, latest_run, 'weights', 'best.pt')
    
    return model_path

# Step 1: Enhance the image
input_image_path = '../trial.jpg'  # Replace with your input image path
output_dir = '../trial test results'  # Replace with your desired output directory

# Enhance the image - you can also customize rho and k parameters here
enhance_image_from_user(input_image_path, output_dir, rho=0.5, k=1.5)

# Get the path to the enhanced image
enhanced_image_path = os.path.join(output_dir, os.path.basename(input_image_path))

# Step 2: Get the latest model path
model_dir = "C:/Users/pauli/DeepDent/periodontitis/notebooks/runs/segment"  # Your base model directory
latest_model_path = get_latest_model(model_dir)

# Load the YOLOv8 segmentation model
model = YOLO(latest_model_path)

# Run inference on the enhanced image
results = model(enhanced_image_path)

# Display the image with segmentation results 
for result in results:
    
    result.show(labels=False, conf=False)  # No labels, no confidence scores
    
    # Optionally, save the output without bounding boxes and labels
    result.save(filename="output.jpg", labels=False, conf=False)

# Optionally, load and display the saved image using OpenCV
output_image = cv2.imread("output.jpg")
plt.imshow(cv2.cvtColor(output_image, cv2.COLOR_BGR2RGB))
plt.axis("off")
plt.show()

Enhanced image saved to ../trial test results\trial.jpg

image 1/1 C:\Users\pauli\DeepDent\periodontitis\notebooks\..\trial test results\trial.jpg: 320x640 29 Individual Tooths, 175.2ms
Speed: 3.1ms preprocess, 175.2ms inference, 38.4ms postprocess per image at shape (1, 3, 320, 640)


<Figure size 640x480 with 1 Axes>

Comparison of Models: 

With Optimization (ADAMW): 'metrics/precision(B)': 0.9742208254855852, 'metrics/recall(B)': 0.9814814814814815, 'metrics/mAP50(B)': 0.9933031152320513, 'metrics/mAP50-95(B)': 0.7070567895774198, 'metrics/precision(M)': 0.9833121997880452, 'metrics/recall(M)': 0.9814814814814815, 'metrics/mAP50(M)': 0.9936824299088521, 'metrics/mAP50-95(M)': 0.6202035950038778, 'fitness': 1.3932329006372584

Without: 'metrics/precision(B)': 0.9809389992396716, 'metrics/recall(B)': 0.9666666666666667, 'metrics/mAP50(B)': 0.9926364173857323, 'metrics/mAP50-95(B)': 0.7107609576378374, 'metrics/precision(M)': 0.9781189457609283, 'metrics/recall(M)': 0.9629629629629629, 'metrics/mAP50(M)': 0.992354104993544, 'metrics/mAP50-95(M)': 0.5907389132334466, 'fitness': 1.3698489360220836

Comparison of Models: With vs. Without Optimization (ADAMW)

Key Observations:
1. **Precision:**
   - **(B)** With ADAMW: **0.9742**, Without: **0.9809** → Slightly lower with ADAMW.
   - **(M)** With ADAMW: **0.9833**, Without: **0.9781** → Slightly higher with ADAMW.

2. **Recall:**
   - **(B)** With ADAMW: **0.9815**, Without: **0.9667** → Higher with ADAMW.
   - **(M)** With ADAMW: **0.9815**, Without: **0.9630** → Higher with ADAMW.

3. **mAP50 (Mean Average Precision at IoU=0.5):**
   - **(B)** With ADAMW: **0.9933**, Without: **0.9926** → Slightly higher with ADAMW.
   - **(M)** With ADAMW: **0.9937**, Without: **0.9924** → Slightly higher with ADAMW.

4. **mAP50-95 (Mean Average Precision across IoU thresholds 0.5-0.95):**
   - **(B)** With ADAMW: **0.7071**, Without: **0.7108** → Slightly lower with ADAMW.
   - **(M)** With ADAMW: **0.6202**, Without: **0.5907** → Higher with ADAMW.

5. **Fitness Score:**
   - With ADAMW: **1.3932**
   - Without: **1.3698**
   - **ADAMW improves the overall fitness score.**
   
Conclusion:
- **ADAMW Optimization improves recall and fitness while maintaining or slightly improving precision and mAP50.**
- **mAP50-95 (B) sees a slight drop, but (M) improves.**